In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os


In [ ]:
# def train_my_model(market_train_df, news_train_df):
#     pass
# def make_my_predictions(market_obs_df,news_obs_df,predictions_template_df):
#     pass

# from kaggle.competitions import twosigmanews
# env = twosigmanews.make_env()

# (market_train_df, news_train_df) = env.get_training_data()
# train_my_model(market_train_df, news_train_df)

# for (market_obs_df, news_obs_df, predictions_template_df) in env.get_prediction_days():
#   predictions_df = make_my_predictions(market_obs_df, news_obs_df, predictions_template_df)
#   env.predict(predictions_df)

# env.write_submission_file()

### Importing Dataframes

In [ ]:
from kaggle.competitions import twosigmanews
# You can only call make_env() once, so don't lose it!
env = twosigmanews.make_env()
print('Done!')

In [ ]:
(market_train_df, news_train_df) = env.get_training_data()

In [ ]:
market_train_df.head()

In [ ]:
news_train_df.head()

### Data Cleaning

### Text Processing

### One Hot Encoding

In [ ]:
def one_hot_encoder(data, nan_as_category=True):
    original_columns = list(data.columns)
    categorical_columns = [col for col in data.columns \
                           if not pd.api.types.is_numeric_dtype(data[col].dtype)] # ignore error in eclipse
    for c in categorical_columns:
        if nan_as_category:
            data[c].fillna('NaN', inplace = True)
        values = list(data[c].unique())
        for v in values:
            data[str(c) + '_' + str(v)] = (data[c] == v).astype(np.uint8)
    data.drop(categorical_columns, axis = 1, inplace = True)
    return data, [c for c in data.columns if c not in original_columns]

In [ ]:
one_hot_encoder(market_train_df)
one_hot_encoder(news_train_df)

### Using sklearn for modelling

In [ ]:
y_train = market_train_df['returnsOpenNextMktres10']
X_train = market_train_df[f for f in market_train_df.columns if f != 'returnsOpenNextMktres10']

In [ ]:
%%time

# set parameters (from NYC Taxi Fare)
model = RandomForestRegressor(n_estimators=25, 
                              max_features=20, 
                              max_depth=25, 
                              min_samples_split=3,
                              min_samples_leaf=2, 
                              random_state=0)

# train and get MSE.
model.fit(X_train, y_train)
train_pred = model.predict(X_train)
print(np.sqrt(mean_squared_error(y_train,train_pred)))

### Making Predictions

In [ ]:
for (market_obs_df, news_obs_df, predictions_template_df) in env.get_prediction_days(): # Looping over days from start of 2017 to 2019-07-15
# market_obs_df always have time values of 22:00 UTC. This is a snapshot of the trading day.
# news_obs_df contains news from the previous market day up until the current day 22:00 UTC.
  
    curr_df = market_obs_df.iloc[0] # You are here in time
    predictions_template_df['confidenceValue'] = model.predict(curr_df)
    env.predict(predictions_template_df) # Saves your predictions for the day


### Export Submission

In [ ]:
env.write_submission_file() # Writes your submission file

### Sources:
* [Getting Started - DJ Sterling](https://www.kaggle.com/dster/two-sigma-news-official-getting-started-kernel)
* [Bare bones script - William Cukierski](https://www.kaggle.com/wcukierski/bare-bones-script-loop-with-comments)
* [Extra data - aaron7sun](https://www.kaggle.com/aaron7sun/stocknews)
* [Text Preprocessing - Andrew Gelé](https://www.kaggle.com/ndrewgele/omg-nlp-with-the-djia-and-reddit)
* [fake news - SamLloyd](https://www.kaggle.com/sjdlloyd/it-s-fake-news-this-is-top-of-the-leaderboard)